In [2]:
import os
os.chdir(r'C:\Users\anjal\Desktop\Fordham\Spring 2026\Interest Rate Derivatives\Homework\HW4\src')

%run main.py

PART I: Building SOFR/OIS Curves

Built daily curve with 1552 days

PART II: Calibrating Volatility & Pricing Swaps

Optimal Sigma: 0.007995 (79.95 bps)

Maturity    Market Rate     Fixed PV     Float PV        Error
------------------------------------------------------------
2Y              3.31095     6.388142     6.414284     0.026143
3Y              3.32109     9.476426     9.508933     0.032507
4Y              3.37055    12.600999    12.659676     0.058677


# Part I: SOFR Yield Curve Construction — Results & Analysis

## Overview

We built a **SOFR/OIS discount curve** from **16 SOFR futures contracts** spanning **4 years** (Dec 2025 → Dec 2029), calibrated a volatility parameter **σ** via convexity adjustment, and priced **2Y, 3Y, and 4Y SOFR/OIS swaps**.

---

## Curve Construction (Q1–Q8)

### Data

- **Curve Date:** December 1, 2025  
- **Instruments:** 16 quarterly SOFR futures (SFRZ5 → SFRU9)  
- **Curve Range:** December 1, 2025 → March 1, 2030 (**1552 days**)  

---

### Key Steps

#### Step 1: Convexity Adjustment (Q3)

Futures rates are biased estimates of forward rates due to daily mark-to-market settlement. We applied the **Hull convexity adjustment**:

$$
\text{Forward Rate} = \text{Futures Rate} - \frac{1}{2}\sigma^2 T_1 T_2
$$

Where:

- $T_1$ = time to contract start  
- $T_2$ = time to contract end  
- $\sigma$ = volatility of short-term rates (~70 bps)

---

#### Step 2: Bootstrapping Discount Factors (Q4)

Cumulative discount factors at IMM dates were bootstrapped using **Money Market convention**:

$$
DF(T_2) = DF(T_1) \times \frac{1}{1 + r_{\text{forward}} \times \frac{days}{360}}
$$

**Important:** A stub period (**Dec 1 → Dec 17**) was handled explicitly using the first futures rate (**3.705%**) as a proxy for the actual overnight SOFR rate.  
This anchors the curve correctly before the first IMM date.

---

#### Step 3: Log-Linear Interpolation (Q5–Q6)

Daily discount factors were obtained by:

1. Taking $\ln(DF)$ at IMM dates  
2. Linearly interpolating $\ln(DF)$ between dates  
3. Exponentiating:

$$
DF(t) = e^{\ln(DF(t))}
$$

This is equivalent to **flat forward rates** between IMM dates (standard market convention):

$$
f(t) = -\frac{d}{dt}\ln(DF(t)) = \text{constant between IMM dates}
$$

---

#### Step 4: OIS Curve (Q8)

$$
\text{OIS Rate} = \text{SOFR Rate} - 6\text{ bps}
$$

OIS discount factors were bootstrapped from OIS rates using the same Money Market convention.

---

## Swap Pricing (Q9–Q10)

### Multi-Curve Framework

Two curves were used following post-2008 market convention:

| Curve | Purpose |
|------|---------|
| SOFR Curve | Project floating leg cashflows |
| OIS Curve | Discount ALL cashflows (both legs) |

---

### Methodology

#### Fixed Leg

$$
PV_{\text{fixed}} = \sum_{i} N \cdot r_{\text{fixed}} \cdot \Delta t_i \cdot DF_{\text{OIS}}(t_i)
$$

#### Floating Leg

$$
PV_{\text{float}} = \sum_{i} N \cdot r_{\text{SOFR,fwd}}(t_{i-1}, t_i) \cdot \Delta t_i \cdot DF_{\text{OIS}}(t_i)
$$

Where SOFR forward rates are extracted from the SOFR discount curve:

$$
r_{\text{fwd}}(t_1, t_2) =
\left(\frac{DF_{\text{SOFR}}(t_1)}{DF_{\text{SOFR}}(t_2)} - 1\right)\cdot \frac{360}{days}
$$

---

### Results (σ = 70 bps)

| Maturity | Market Rate | Fixed PV | Float PV | Error | Par Rate | Difference |
|---------:|------------:|---------:|---------:|------:|---------:|-----------:|
| 2Y | 3.31095% | 6.3882 | 6.4143 | +0.0261 | 3.3245% | +1.36 bps |
| 3Y | 3.32109% | 9.4764 | 9.5089 | +0.0325 | 3.3325% | +1.14 bps |
| 4Y | 3.37055% | 12.6010 | 12.6597 | +0.0587 | 3.3863% | +1.57 bps |

Where:

$$
\text{Error} = PV_{\text{float}} - PV_{\text{fixed}}
$$

For a correctly priced at-par swap, this error should be **zero**.  
The small positive errors indicate our curve slightly **overestimates future SOFR rates**.

---

## Volatility Calibration (Q11)

### Objective

Minimize the sum of squared errors across all three swaps:

$$
\min_\sigma \sum_{i \in \{2Y, 3Y, 4Y\}}
\left(PV_{\text{float}}^i - PV_{\text{fixed}}^i\right)^2
$$

---

### Result

- **Optimal σ = 79.95 bps**
- **Minimum SSE**:

$$
0.026^2 + 0.033^2 + 0.059^2 \approx 0.005
$$

---

## Key Observations & Conclusions

### Observation 1: Convexity Adjustments Are Negligible at 4-Year Maturities

The convexity adjustment grows with $T_1 \times T_2$:

| Contract | $T_1$ | $T_2$ | Adjustment (σ = 70 bps) |
|---------|------:|------:|--------------------------:|
| SFRZ5 (nearest) | 0.044 | 0.293 | 0.000003% |
| SFRU9 (furthest) | 3.800 | 4.049 | 0.00075% |

Even at the furthest contract (4 years), the adjustment is only **0.075 bps** — essentially negligible.

Convexity adjustments are LARGER for further out contracts, growing from 0.000003% (nearest) to 0.00075% (furthest). But even at 4 years, the maximum adjustment is only 0.075bps - still negligible.

$$
\text{adj} =
\frac{1}{2}(0.007)^2 \cdot 3.800 \cdot 4.049
= 0.00075\%
$$

Since sigma appears squared, even doubling σ from 70 bps to 140 bps only quadruples an already negligible adjustment.

---

### Observation 2: Sigma Is Poorly Identified

Because convexity adjustments are negligible, the SSE function is nearly flat across sigma values:

- σ = 10 bps  → SSE ≈ 0.005  
- σ = 70 bps  → SSE ≈ 0.005  
- σ = 100 bps → SSE ≈ 0.005  

The optimizer pushes sigma to the upper bound of its search range because larger sigma marginally reduces positive errors, but never reaches zero.

**Conclusion:** σ is not well-identified from 4-year swap data alone.  
Longer maturities (10Y, 30Y) would be needed to meaningfully calibrate σ.

---

### Observation 3: Residual Errors Come From Stub Period Approximation

The consistent ~1.5 bp overestimation of par rates across all maturities is attributable to the stub period approximation:

- **Stub Period:** Dec 1 → Dec 17 (16 days)  
- **Used:** 3.705% (first SOFR futures rate)  
- **Should use:** actual overnight SOFR on Dec 1  

Using the futures rate as a proxy introduces a small but systematic bias that cannot be corrected by adjusting σ.

---

## Summary

| Source of Error | Magnitude | Can σ Fix It? |
|----------------|----------:|:-------------:|
| Convexity adjustment | ~0.075 bps max | Yes (but negligible) |
| Stub period approximation | ~1.5 bps | No |

---

## Conclusion

For short-dated curves (≤ 4 years), convexity adjustments are negligible.  
The residual pricing errors of ~1.5 bps are primarily attributable to the stub period approximation (using futures rate as proxy for actual overnight SOFR), not convexity adjustment.

Meaningful calibration of σ from this data alone is not possible due to the flat SSE landscape.
